In [ ]:
from PIL import Image
import socket
import time
import torch
import cv2
import numpy as np
from torchvision import transforms
import pickle as pkl

from utils.datasets import letterbox
from utils.general import non_max_suppression_kpt
from utils.plots import output_to_keypoint, plot_skeleton_kpts



In [ ]:
# Change forward pass input size.
input_size = 960

# Select the device based on hardware configs.
if torch.cuda.is_available():
    device = torch.device("cuda:0")
else:
    device = torch.device("cpu")
print('Selected Device : ', device)

# Load keypoint detection model.
weights = torch.load('yolov7-w6-pose.pt', map_location=device)
model = weights['model']
# Load the model in evaluation mode.
_ = model.float().eval()
# Load the model to computation device [cpu/gpu/tpu]
model.to(device)

# Webcam capture
cap = cv2.VideoCapture(0)  # 0 corresponds to the default camera (usually the built-in webcam)

# May need to change the w, h as letterbox function reshapes the image.
w = 1920#int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
h = 1080#int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

# Video writer initialization
out = cv2.VideoWriter('pose_outputs/webcam_output.mp4',
                      cv2.VideoWriter_fourcc(*'mp4v'),
                      30, (w, h))

import pygame
import sys
import random
import math

# Initialize Pygame
pygame.init()

# Constants
WIDTH, HEIGHT = 960, 768
INITIAL_BALL_SPEED = 20
BALL_SPEED = INITIAL_BALL_SPEED
BALL_ACCELERATION = 1.001
PADDLE_SPEED = 8
WHITE = (255, 255, 255)
FPS = 60

# Initialize screen
screen = pygame.display.set_mode((WIDTH, HEIGHT))
pygame.display.set_caption("Pong Game")

# Initialize clock
clock = pygame.time.Clock()

# Initialize fonts
font = pygame.font.Font(None, 36)

# Initialize variables
ball = pygame.Rect(WIDTH // 2 - 15, HEIGHT // 2 - 15, 30, 30)
ball_speed = [random.choice([-1, 1]) * INITIAL_BALL_SPEED, random.uniform(-1, 1) * INITIAL_BALL_SPEED]

player1 = pygame.Rect(20, HEIGHT // 2 - 50, 10, 100)
player2 = pygame.Rect(WIDTH - 30, HEIGHT // 2 - 50, 10, 100)

score_player1 = 0
score_player2 = 0

ball_scored = False  # Flag to prevent multiple scoring for the same event
game_over = False  # Flag to indicate whether the game is over

def reset_game():
    global ball, ball_speed, ball_scored, game_over
    ball = pygame.Rect(WIDTH // 2 - 15, HEIGHT // 2 - 15, 30, 30)
    ball_speed = [random.choice([-1, 1]) * INITIAL_BALL_SPEED, random.uniform(-1, 1) * INITIAL_BALL_SPEED]
    ball_scored = False

def draw_text(text, x, y):
    text_surface = font.render(text, True, WHITE)
    text_rect = text_surface.get_rect(center=(x, y))
    screen.blit(text_surface, text_rect)


In [16]:

while cap.isOpened():
    BALL_SPEED *= BALL_ACCELERATION
    ret, frame = cap.read()

    if not ret:
        print('Unable to read frame. Exiting ..')
        break

    mapped_img = frame.copy()
    # Letterbox resizing.
    img = letterbox(frame, input_size, stride=64, auto=True)[0]
    #print(img.shape)
    img_ = img.copy()
    # Convert the array to 4D.
    img = transforms.ToTensor()(img)
    # Convert the array to Tensor.
    img = torch.tensor(np.array([img.numpy()]))
    # Load the image into the computation device.
    img = img.to(device)
    pil_img = Image.fromarray(frame)
    # Gradients are stored during training, not required while inference.
    with torch.no_grad():
        t1 = time.time()
        output, _ = model(img)
        
        
        t2 = time.time()
        fps = 1/(t2 - t1)
        output = non_max_suppression_kpt(output, 
                                         0.25,    # Conf. Threshold.
                                         0.65,    # IoU Threshold.
                                         nc=1,   # Number of classes.
                                         nkpt=17, # Number of keypoints.
                                         kpt_label=True)
        output2key = output_to_keypoint(output)
        
        
    points = np.zeros((output2key.shape[0], 17, 2))
    
    for entity in range(output2key.shape[0]):
        keypoints = output2key[entity, 7:].T
        for kpt_idx in range(17):  # Assuming you have 17 keypoints
            idx = kpt_idx * 3  # Each keypoint has x, y, and confidence values
            x, y, conf = keypoints[idx:idx+3]
            y = max(0, min(1, (y-150)/568))
            x = max(0, min(1, (x-50)/960))
            if conf > 0.6:
                points[entity, kpt_idx] = [x,  y]
            else:
                points[entity, kpt_idx] = [-1, -1]
    # Change format [b, c, h, w] to [h, w, c] for displaying the image.
    nimg = img[0].permute(1, 2, 0) * 255
    nimg = nimg.cpu().numpy().astype(np.uint8)
    nimg = cv2.cvtColor(nimg, cv2.COLOR_RGB2BGR)
    
    for idx in range(output2key.shape[0]):
        plot_skeleton_kpts(nimg, mapped_img, input_size, output2key[idx, 7:].T, 3)
    cv2.putText(nimg, 'FPS : {:.2f}'.format(fps), (200, 40), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2,
                cv2.LINE_AA)
    cv2.putText(nimg, 'YOLOv7', (20, 40), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)

    cv2.imshow('Output', nimg[..., ::-1])
    out.write(nimg[..., ::-1])
    key = cv2.waitKey(1)
    if key == ord('q'):
        break
            
            
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            pygame.quit()
            sys.exit()

    keys = pygame.key.get_pressed()

    # Check if the game is over
    if not game_over:
        # Update player1 position
        #player1.y += (keys[pygame.K_s] - keys[pygame.K_w]) * PADDLE_SPEED
        if len(points) > 0:
            if points[0, 9, 1] != -1:
                player1.y = points[0, 9, 1] * HEIGHT
        else:
            player1.y = 0
        player1.y = max(0, min(player1.y, HEIGHT - player1.height))

        # Update player2 position
        #player2.y += (keys[pygame.K_s] - keys[pygame.K_w]) * PADDLE_SPEED
        if len(points) > 0:
            if points[0, 10, 1] != -1:
                player2.y = points[0, 10, 1] * HEIGHT
        else:
            player2.y = 0
        player2.y = max(0, min(player2.y, HEIGHT - player2.height))

        # Ball movement
        ball.x += ball_speed[0]
        ball.y += ball_speed[1]

        # Ball collisions
        if ball.colliderect(player1) or ball.colliderect(player2):
            ball_speed[0] = -ball_speed[0]
            ball_speed[1] = ball_speed[1] + random.uniform(-.1, .1) * BALL_SPEED

        if ball.top <= 0 or ball.bottom >= HEIGHT:
            ball_speed[1] = -ball_speed[1]

        # Score update
        if ball.left <= 0:
            if not ball_scored:
                score_player2 += 1
                ball_scored = True
                if score_player2 == 10:
                    game_over = True

        elif ball.right >= WIDTH:
            if not ball_scored:
                score_player1 += 1
                ball_scored = True
                if score_player1 == 10:
                    game_over = True

        # Reset the ball when a point is scored
        if ball.left <= 0 or ball.right >= WIDTH:
            reset_game()

        # Reset ball_scored flag when ball crosses the center line
        if WIDTH // 2 - 5 < ball.centerx < WIDTH // 2 + 5:
            ball_scored = False


    # Increase ball speed over time
    ball_speed[0] *= 1.0001
    ball_speed[1] *= 1.0001

    # Draw everything
    screen.fill((0, 0, 0))
    pygame.draw.rect(screen, WHITE, player1)
    pygame.draw.rect(screen, WHITE, player2)

    if not game_over:  # Draw the ball only if the game is not over
        pygame.draw.ellipse(screen, WHITE, ball)

    draw_text(str(score_player1), WIDTH // 4, 50)
    draw_text(str(score_player2), WIDTH // 4 * 3, 50)

    # Check for winner
    if game_over:
        draw_text("Player 1 Wins!" if score_player1 == 10 else "Player 2 Wins!", WIDTH // 2, HEIGHT // 2)
        draw_text("Press SPACE to play again", WIDTH // 2, HEIGHT // 2 + 50)

        keys = pygame.key.get_pressed()
        if keys[pygame.K_SPACE]:
            reset_game()
            game_over = False
            score_player1 = 0
            score_player2 = 0

    pygame.display.flip()
    clock.tick(FPS)


Selected Device :  cuda:0


SystemExit: 

C:\Users\mettn\miniconda3\envs\pt\Lib\site-packages\IPython\core\interactiveshell.py:3556: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [ ]:
cap.release()
out.release()
cv2.destroyAllWindows()